In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import time
import pickle
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
import torch

print(cv2.__version__)
sift  = cv2.xfeatures2d.SIFT_create()

3.4.1


In [2]:
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

"""
Checking the distribution of labels
in the training set
"""
hist_label = [0]*9
for filename in os.listdir('training_set'):   #training_set  
    photo_id = filename.split('.')
    lst_labels = bus_to_labels_dict[photo_to_bus_dict[photo_id[0]]]
    labels = lst_labels[0].split(' ')
    if len(labels)!=0:
        for i in labels:
            if i is not '':
                hist_label[int(i)]+=1

print("Histogram distribution for the labels in the training_set")
print(hist_label)

Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000
Histogram distribution for the labels in the training_set
[9163, 23679, 25603, 19453, 16829, 29454, 30772, 14825, 20300]


In [3]:
"""
Get the testing data
Returns images and respective photo ids
"""
def get_test_data():
    data = []
    photo_ids = []
    count=0
    for filename in sorted(os.listdir('Bow_test/'),key=lambda x: int(os.path.splitext(x)[0])):
        photo_ids.append(filename.split('.')[0])
        data.append(cv2.imread('Bow_test/'+filename,0))
        count+=1
    return data,photo_ids
    
test_data, test_photo_ids = get_test_data()

In [4]:
"""
Creates no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels

In [5]:
"""
Get the training data
Returns images and respective photo ids
"""
def get_train_data():
    data = []
    photo_ids = []
    count=0
    for filename in sorted(os.listdir('training_set/'),key=lambda x: int(os.path.splitext(x)[0])):
        photo_ids.append(filename.split('.')[0])
        data.append(cv2.imread('training_set/'+filename,0))
        count+=1
    return data,photo_ids
    
train_data, train_photo_ids = get_train_data()

In [6]:
#Training set
"""
Contains no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
train_photo_labels = id_to_label(train_photo_ids)
print(train_photo_labels[0])
print(bus_to_labels_dict[photo_to_bus_dict[train_photo_ids[0]]])
print(train_photo_labels.shape)
print(train_photo_labels[:,0].shape)

[ 0.  1.  1.  0.  1.  1.  1.  1.  0.]
['1 2 4 5 6 7']
(40000, 9)
(40000,)


In [7]:
#Testing set
"""
Contains no_images*labels_size matrix
with each row indicating what labels
are assigned to each photo
"""
test_photo_labels = id_to_label(test_photo_ids)
print(test_photo_labels[0])
print(bus_to_labels_dict[photo_to_bus_dict[test_photo_ids[0]]])
print(test_photo_labels.shape)
print(test_photo_labels[:,0].shape)

[ 0.  0.  0.  0.  0.  0.  1.  0.  1.]
['6 8']
(24, 9)
(24,)


In [8]:
"""
Function that does the extraction of dense SIFT descriptors
params: 
   data:   vector of images       
return:
    temp_des: vector of descriptors for invidual image
    des:      a combined matrix of all images
"""
"""
Parameters to be considered step and scale.
"""
step = 50
scale = 20
def get_dense_descriptors(data):
    temp_des = []
    des = np.empty(shape=[0,128])
    for i in data:
        kp = [cv2.KeyPoint(y, x,scale) for y in range(0, i.shape[0], step) 
                                for x in range(0, i.shape[1], step)]
        kp, dense_feat = sift.compute(i, kp)
        dense_feat = np.array(dense_feat)
        temp_des.append(dense_feat)
        des = np.append(des,dense_feat,axis=0)
    return temp_des, des 

"""
Extracting descriptors for testing data
"""
t0 = time.time()
test_individual_des, test_des = get_dense_descriptors(test_data)
print("Shape of overall test images descriptors:-")
print(test_des.shape)
print("Extracting test images descriptors took %0.3fs." % (time.time() - t0))
print("write descriptors test finished")

Shape of overall test images descriptors:-
(1880, 128)
Extracting test images descriptors took 4.418s.
write descriptors test finished


In [9]:
"""
Generating histograms for train/test
based on the clusters that we get K-Means
"""
def create_histogram(individual_data, kmeans,no_clusters):
    hist = np.empty(shape=[0,no_clusters])
    for i in individual_data:
        pred_te = kmeans.predict(i)
        h = np.histogram(pred_te,bins = np.arange(0,no_clusters+1))
        total = i.shape[0]
        tmp = []
        for j in range(0,no_clusters):
            tmp.append(h[0][j])
        tmp = np.asarray(tmp)
        tmp = tmp.reshape(1, no_clusters)
        tmp = tmp/(total*1.0)
        hist= np.append(hist,tmp,axis=0)
    return hist

In [10]:
#For loading the model
kmeans = pickle.load(open("clusters/bov_pickle_new_50_step_50_scale_20.sav",'rb'))
no_clusters = 50
"""
Creating histograms or Bag of visual words
for training and testing descriptors using K-Means cluster from before
"""
t0 = time.time()
test_hist = create_histogram(test_individual_des, kmeans, no_clusters)
print("Calculating histograms took %0.3fs." % (time.time() - t0))

Calculating histograms took 0.030s.


In [11]:
"""
Training 9 random classifiers sperately for each label as a target
"""
t0 = time.time()

Ypred = []
num_classes = 9
clf = [None]*num_classes
for i in range(num_classes):
    rf = pickle.load(open("random_forest/classifier_" + str(i) + ".pkl",'rb'))
    print("getting predictions for attribute:",i)
    y_pred = rf.predict(test_hist)
    Ypred.append(y_pred)
  
print("Training took %0.3fs." % (time.time() - t0))
print("preparing output...")
Ypred = np.vstack(Ypred)
Ypred = np.transpose(Ypred)
Ypred = Ypred.tolist()

getting predictions for attribute: 0
getting predictions for attribute: 1
getting predictions for attribute: 2
getting predictions for attribute: 3
getting predictions for attribute: 4
getting predictions for attribute: 5
getting predictions for attribute: 6
getting predictions for attribute: 7
getting predictions for attribute: 8
Training took 3.336s.
preparing output...


In [12]:
def verify_correctness(pred1, test_labels, threshold=0):
    cnt=0
    count = 0
    write = 0
    for x, y in zip(pred1, test_labels):
        x1= np.array(x)
        y1 = np.array(y)
        c=0
        count = count + 1
        for a, b in zip(x1, y1):
            if(a!=b): 
                c+=1
        if c<=threshold:
            cnt+=1
    accuracy = cnt/len(test_labels)
    return accuracy

In [13]:
print("Accuracy score for strict match is ",verify_correctness(Ypred,test_photo_labels,0)*100)   #Strict match

Accuracy score for strict match is  75.0


In [14]:
"""
Ensemble method
"""

#Load saved models
nnb1 = joblib.load('models/nnbr1.model')
rfc1 = joblib.load('models/rfc1.model')
dtr1 = joblib.load('models/dtr1.model')
sgd1 = joblib.load('models/sgd1.model')

dist, predictions = nnb1.kneighbors(test_hist)
ensemble_pred1 = []
for each in predictions:
    ensemble_pred1.append(train_photo_labels[each[0]])
ensemble_pred2 = sgd1.predict(test_hist)
ensemble_pred3 = dtr1.predict(test_hist)
ensemble_pred4 = rfc1.predict(test_hist)

/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator NearestNeighbors from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SGDClassifier from version

In [15]:
print("Accuracy score for strict match (Nearest Neigbhor) is ",verify_correctness(ensemble_pred1,test_photo_labels,1)*100)   #Strict match
print("Accuracy score for strict match (SGDClassifier) is ",verify_correctness(ensemble_pred2,test_photo_labels,1)*100)   #Strict match
print("Accuracy score for strict match (Decision Tree) is ",verify_correctness(ensemble_pred3,test_photo_labels,1)*100)   #Strict match
print("Accuracy score for strict match (Random Forest) is ",verify_correctness(ensemble_pred4,test_photo_labels,1)*100)   #Strict match

Accuracy score for strict match (Nearest Neigbhor) is  25.0
Accuracy score for strict match (SGDClassifier) is  37.5
Accuracy score for strict match (Decision Tree) is  16.666666666666664
Accuracy score for strict match (Random Forest) is  29.166666666666668


In [16]:
"""
ResNet   
"""
import cv2
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import csv
import time
import pickle
import glob

#For Res-Net matrix creation
def get_all_img_in_tensor(img, prep):
    img_list = []
    img1 = img[:224, :224]
    img_list.append(prep(img1))
    img2 = img[:224, 151:]
    img_list.append(prep(img2))
    img3 = img[138:362, :224]
    img_list.append(prep(img3))
    img4 = img[138:362, 151:]
    img_list.append(prep(img4))
    img5 = img[276:, :224]
    img_list.append(prep(img5))
    img6 = img[276:, 151:]
    img_list.append(prep(img6))
    img7 = img[138:362, 75:299]
    img_list.append(prep(img7))
    return img_list

def initialize_model():
    model_ft = models.resnet50(pretrained=True)
    input_size = 224
    return model_ft, input_size

# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model, size = initialize_model()
model = model.to(device)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])

#Matrix creation for Res-Net
def get_img_in_tensor(img, prep):
    img_list = []
    resized_img = cv2.resize(img, (224, 224))
    img_list.append(prep(resized_img))
    img2 = resized_img
    img_list.append(prep(img2))
    return img_list

inputpath = 'demo_testset/'
outpath = 'demo_gendata/'
folder_list = sorted(os.listdir(inputpath))
try:
    folder_list. remove('.DS_Store')
except:
    pass

i = 1
out_list = np.empty(shape=[0,4096])
for file_name in sorted(os.listdir(inputpath)):
    img = cv2.imread(inputpath + file_name)
    if img.shape != (500,375,3):
        continue
    img_list = get_all_img_in_tensor(img, prep)    ## avg of 7 images
    inp = torch.stack(img_list)
    inp = inp.to(device)
    outputs = model(inp)
    mean_output = outputs.mean(0)
    np_out = mean_output.cpu().detach().numpy()
    i = i + 1
    #write the out_list to file
    print("saving the file ", file_name[:-4]+'.resnet')
    sio.savemat(outpath + file_name[:-4] +'.resnet', {'features':np_out}, do_compression=True)
    
"""
Get the testing data
Returns images and respective photo ids
"""
def get_test_data():
    data = []
    photo_ids = []
    count=0
    for filename in sorted(os.listdir('demo_testset/'),key=lambda x: int(os.path.splitext(x)[0])):
        if count%5000==4999:
            print(filename)
        photo_ids.append(filename.split('.')[0])
        data.append(cv2.imread('demo_testset/'+filename,0))
        count+=1
    return data,photo_ids
    
test_data, test_photo_ids = get_test_data()
print(len(test_data))

"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))

def id_to_label(photo_ids):
    photo_labels = np.empty(shape=[0,9])
    for id in photo_ids:
        lst_labels = bus_to_labels_dict[photo_to_bus_dict[id]]
        ls = lst_labels[0].split(' ')
        labels = [0]*9
        for label in ls:
            if label !='':
                labels[int(label)] = 1
        labels = np.array(labels).reshape(1,-1)
        photo_labels = np.append(photo_labels,labels,axis=0)
    return photo_labels

test_photo_labels = id_to_label(test_photo_ids)
print(bus_to_labels_dict[photo_to_bus_dict[test_photo_ids[0]]])
print(test_photo_labels[:,0].shape)

mat_tst=[]
paths = glob.glob('demo_gendata/*.mat')
for matrix in paths:
    mat=sio.loadmat(matrix)
    m = mat['features'].reshape(-1)
    mat_tst.append(m)

resnet1 = joblib.load('models/resnet1.model')
resnet2 = joblib.load('models/resnet2.model')

resnet1_pred = resnet1.predict(mat_tst)
resnet2_pred = resnet2.predict(mat_tst)

GPU is active
saving the file  3184.resnet
saving the file  3196.resnet
saving the file  3216.resnet
saving the file  3220.resnet
saving the file  3228.resnet
saving the file  3235.resnet
saving the file  3238.resnet
saving the file  3239.resnet
saving the file  3290.resnet
37
Column names are photo_id, business_id
Processed 234843 lines.
234842
Column names are business_id, labels
Processed 2001 lines.
2000
['1 4 5 6 7']
(37,)


/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearSVC from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/akshay_mallipeddi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MultiOutputClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [17]:
print("Accuracy score for strict match is ",verify_correctness(resnet1_pred,test_photo_labels,2)*100)   #Strict match
print("Accuracy score for strict match is ",verify_correctness(resnet2_pred,test_photo_labels,2)*100)   #Strict match

Accuracy score for strict match is  13.513513513513514
Accuracy score for strict match is  10.81081081081081
